In [ ]:
import os
import shutil
import tensorflow as tf
from tensorflow import keras
from keras.applications import ResNet50, InceptionV3, InceptionResNetV2, DenseNet121
from keras.layers import GlobalAveragePooling2D, Dense, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE
import numpy as np
from collections import Counter
from keras.losses import SparseCategoricalCrossentropy
from google.colab import drive
import matplotlib.pyplot as plt

drive.mount('/content/drive')

# Parameters and hyperparameters
batch_size = 32
epochs = 150

# ReduceLROnPlateau
min_lr = 1e-10
factor = 0.2

# Optimizer
learning_rate = 0.001

# Early Stopping & Scheduler
patience = 10

# Early Stopping
start_epoch = 50
min_delta= 0.001

# Classes
class_names = ['front_bunched', 'front_up', 'mid_bunched', 'tip_up']
num_classes = len(class_names)

# Image Size
image_size = (256,256)

# Directories
data_dir = '/content/drive/MyDrive/EM401/SMOTE/Dataset'

# Load input data
def load_data(data_dir):
    X, y = [], []
    for class_name in os.listdir(data_dir):
        class_dir = os.path.join(data_dir, class_name)
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            img = tf.keras.preprocessing.image.load_img(img_path, target_size=(256, 256))
            img = tf.keras.preprocessing.image.img_to_array(img)
            X.append(img)
            y.append(class_name)
    X = np.array(X)
    y = np.array(y)
    return X, y

# Split the dataset first
X, y = load_data(data_dir)
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42) # 0.25 x 0.8 = 0.2

# Apply SMOTE only on the training data
smote = SMOTE()
X_train_resampled, y_train_resampled = smote.fit_resample(X_train.reshape(X_train.shape[0], -1), y_train)

# Normalize the data
X_train_resampled = X_train_resampled.reshape(-1, 256, 256, 3) / 255.0
X_val = X_val / 255.0
X_test = X_test / 255.0

# Convert class names to integer class indices
class_indices = {class_name: i for i, class_name in enumerate(class_names)}
y_train_resampled = np.array([class_indices[label] for label in y_train_resampled])
y_val = np.array([class_indices[label] for label in y_val])
y_test = np.array([class_indices[label] for label in y_test])

# Load base model
wResNet50 = '/content/drive/MyDrive/EM401/models/RadImageNet_models/RadImageNet-ResNet50_notop.h5'
wIRV2 = '/content/drive/MyDrive/EM401/models/RadImageNet_models/RadImageNet-IRV2_notop.h5'
wInceptionV3 = '/content/drive/MyDrive/EM401/models/RadImageNet_models/RadImageNet-InceptionV3_notop.h5'
wDenseNet121 = '/content/drive/MyDrive/EM401/models/RadImageNet_models/RadImageNet-DenseNet121_notop.h5'

base_model = DenseNet121(weights=wDenseNet121, include_top=False, input_shape=(256, 256, 3))

for layer in base_model.layers:
    layer.trainable = False

# Create the model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Set up callbacks
logdir = '/content/drive/MyDrive/EM401/logs'
os.makedirs(logdir, exist_ok=True)
tensorboard_callback = TensorBoard(log_dir=logdir)
early_stopping = EarlyStopping(monitor='val_loss', min_delta=min_delta, patience=patience, verbose=1, restore_best_weights=True, start_from_epoch=start_epoch)
lr_scheduler = ReduceLROnPlateau(factor=factor, min_lr=min_lr, patience=patience//2, verbose=1)

# Train model
history = model.fit(
    X_train_resampled, y_train_resampled,
    epochs=epochs,
    validation_data=(X_val, y_val),
    batch_size=batch_size,
    callbacks=[tensorboard_callback, early_stopping, lr_scheduler]
)

# Evaluate model on test set
evaluation = model.evaluate(X_test, y_test)
print("Test Accuracy:", evaluation[1])
print("Test Loss:", evaluation[0])

Save model
save_directory = '/content/drive/MyDrive/EM401/models'
os.makedirs(save_directory, exist_ok=True)
model.save(os.path.join(save_directory, 'final_model_smote.h5'))

In [ ]:
# Plot training and validation metrics
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')

plt.tight_layout()
plt.show()

In [ ]:
model.summary()

In [ ]:
# Plot training and validation metrics
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))

# Plot loss
plt.plot(history.history['loss'], label='Training Loss', color='blue')
plt.plot(history.history['val_loss'], label='Validation Loss', color='red')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc='upper left')
plt.title('Training and Validation Loss')

# Create a secondary y-axis for accuracy
plt.twinx()
plt.plot(history.history['accuracy'], label='Training Accuracy', color='green')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', color='orange')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

plt.tight_layout()
plt.show()



In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Predict labels for the test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_classes)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

In [ ]:
from sklearn.metrics import classification_report

# Generate classification report
report = classification_report(y_test, y_pred_classes, target_names=class_names)

# Print classification report
print("Classification Report:")
print(report)

In [ ]:
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import seaborn as sns
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


# Load input data
def load_data(data_dir):
    X, y = [], []
    for class_name in os.listdir(data_dir):
        class_dir = os.path.join(data_dir, class_name)
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            img = tf.keras.preprocessing.image.load_img(img_path, target_size=(256, 256))
            img = tf.keras.preprocessing.image.img_to_array(img)
            X.append(img)
            y.append(class_name)
    X = np.array(X)
    y = np.array(y)
    return X, y

# Parameters and hyperparameters
batch_size = 32
epochs = 150

# Classes
class_names = ['front_bunched', 'front_up', 'mid_bunched', 'tip_up']
num_classes = len(class_names)

# Directories
data_dir = '/content/drive/MyDrive/EM401/Images_Database_Unseen_18_03'

# Load the data
X, y = load_data(data_dir)



# Iterate through each class folder
for class_name in class_names:
    class_dir = os.path.join(data_dir, class_name)
    print(f"Folder: {class_name}")

    # Iterate through images in the folder
    for img_name in os.listdir(class_dir):
        img_path = os.path.join(class_dir, img_name)

        # Load and preprocess the image
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=(256, 256))
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = img_array / 255.0

        # Make predictions
        predictions = model.predict(img_array)
        predicted_class = np.argmax(predictions)

        print(f"Image: {img_name}, Prediction: {class_names[predicted_class]}")
